In [1]:
from fyp_work.paper_to_equation.Gemini.GeminiPredict import GeminiPredict
from sympy import *
import numpy as np

In [2]:
api_key = "GEMINI_API_KEY"
filename = "Data/HTC_gemini4"
model = "tunedModels/mmltopython4-f3fuppiemnq9"

gp = GeminiPredict(api_key, filename, model)
gp.predict_from_file(filename="Data/HTC_mml") 
gp.save_predictions()

API key loaded successfully!


Generating Predictions:   0%|          | 0/13 [00:00<?, ?row/s]

Generating Predictions: 100%|██████████| 13/13 [00:19<00:00,  1.53s/row]


In [43]:
class SystemOfEquations():
    def __init__(self, filepath):
        self.str_equations = []
        self.eq = []
        self.filepath = filepath
        self.symbol_lines = set() # Only keep one instance of each symbol line
        self.symbols = set() # Same for symbols
        self._extract_equations()
        for eq in self.str_equations:
            if not self.validate(eq):
                raise Exception("MathML to SymPy conversion error.") 
        

    def _extract_equations(self):
        """
        Store the equations in a list of strings
        """
        with open (f"{self.filepath}", "r") as file: 
            for line in file:
                self.str_equations.append(line)

    def get_equations(self):
        """
        Getter for the equation list
        """
        if not self.str_equations:
            self.extract_equations()

        return self.str_equations

    def validate(self, equation):
        """
        Check if the equation string is valid SymPy/Python code

        Args:
            equation (str): The equation string to validate

        Returns:
            True if the equation is valid
            False if the equation is invalid
        """
        try:
            eval(equation) # Execute the equation string as code
            return True
        except Exception as e:
            print(e)
            return False

    def parse_equation(self, equation):
        """
        Parse the equation string and store the symbols and equations

        Args:
            equation (str): The equation string to parse
        """
        for line in equation.split("\\n"): # Split equation string into lines (e.g. Symbol, Symbol, Eq)
            if "Symbol" in line:
                line = line.replace('"', "")
                self.symbol_lines.add(line) # Store line for file creation
                self.symbols.add(line.split(" = ")[0]) # Store symbol as a variable
            elif "Eq" in line:
                eq = line.split(" = ")[1]
                eq = eq.replace('"\n', "")
                self.eq.append(sympify(eq)) # Store the equation as a SymPy equation

    def solve(self, target):
        pass
    
    def solve_test(self, target):
        symbol_list = {"a", "b", "c"}
        const = {"c"}

        # Define symbols dynamically
        sd = {name: Symbol(name) for name in symbol_list if name not in const}

        # Substitute c = 4 directly in the equations
        c = 4
        # og_eq = [Eq(a, b + 5), Eq(b, c + 1)]
        equations = [Eq(sd["a"], sd["b"] + 5), Eq(sd["b"], c + 1)]
        # equations = [eq.subs({a: sd["a"], b: sd["b"]}) for eq in og_eq]

        # Solve for a
        result = solve(equations)
        print(result[sd[target]])
    
    def solve_test2(self, target):
        pass





def main():
    soe = SystemOfEquations("Data/HTC_gemini5_predictions.txt")
    for eq in soe.str_equations:
        soe.parse_equation(eq)
    print(soe.eq)
    


main()

[Eq(h, h_c + h_g), Eq(h, 1.45*k*(p/H)**0.985*tan(θ)/σ), Eq(h, 8000*λ*(K*p/(C*σ_U))**0.86), Eq(h, A*(1 - exp(-B*P))), Eq(h, 2*k_f*k_t*k_w*(1 - A)/(h_f*(-k_f*k_t - k_f*k_w + 2*k_t*k_w))), Eq(h, h_a + h_c + h_l), Eq(h_c, K_st*N_P*α/R), Eq(K_st, 2/(1/k_t + 1/k_s)), Eq(R, sqrt(Rs**2 + Rt**2)), Eq(N_P, 1 - exp(-P*λ/σ_U)), Eq(h_l, K_stl*N_L*β/R), Eq(K_stl, 3/(1/k_t + 1/k_s + 1/k_l)), Eq(N_L, 1 - exp(-γ*δ))]


In [ ]:
from collections import defaultdict, deque

class EquationGraph():
    def __init__(self, equations, equation_n):
        self.n = equation_n - 1
        self.equations = equations
        self.graph = defaultdict(set)
        self.var_equation = {}
        self._graph_init()

    def _graph_init(self):
        target_eq = self.equations[self.n]
        target = target_eq.lhs.free_symbols

        duplicates = []
        for i, eq in enumerate(self.equations):
            if eq.lhs.free_symbols == target:
                duplicates.append(i)

        result = [item for i, item in enumerate(self.equations) if i not in duplicates]
        self.equations = result
        self.equations.insert(0, target_eq)
        self.target = target

        for eq in self.equations:
            lhs = eq.lhs.free_symbols
            rhs = eq.rhs.free_symbols

            for var in lhs:
                self.graph[var].update(rhs)
                self.var_equation[var] = eq
    
    def BFS_for_vars(self):
        seen = set()
        queue = deque(self.target)

        while queue:
            var = queue.popleft()
            if var not in seen:
                seen.add(var)
                queue.extend(self.graph[var])
        
        return seen
    
    def get_system_of_equations(self):
        dependencies = self.BFS_for_vars()
        return [self.var_equation[var] for var in dependencies]





def test():
    # Constants
    # k_s (unit=(kW/mK))
    k_s = 0.14
    # k_t (unit=(H13))
    k_t = 0.0244
    # k_t (unit=(Cast iron))
    k_t = 0.044
    # k_t (unit=(P20))
    k_t = 0.0315
    # k_l (unit=(Lubricant))
    k_l = 0.024
    # R_s (unit=(m))
    R_s = 3.4e-07
    Rs = 3.4e-07
    # R_t (unit=(P20))
    R_t = 9.6e-07
    Rt = 9.6e-07
    # h_a (unit=(kW/m^(2)K))
    h_a = 0.8
    # sigma_U (unit=Unknown)
    sigma_U = 21.0
    σ_U = 21.0
    # alpha (unit=(−))
    alpha = 0.000201
    α = 0.000201
    # lambda_ (unit=(−))
    lambda_ = 6.05
    λ = 6.05
    # beta (unit=(−))
    beta = 0.00011
    β = 0.00011
    # gamma (unit=(m^(−1)))
    gamma = 200000.0
    γ = 200000.0
    # delta (unit=(−))
    delta = 1.5e-5
    δ = 1.5e-5

    # Variables
    h = Symbol("h")
    h_c = Symbol("h_c")
    K_st = Symbol("K_st")
    R = Symbol("R")
    N_P = Symbol("N_P")
    h_l = Symbol("h_l")
    K_stl = Symbol("K_stl")
    N_L = Symbol("N_L")

    P = 10
    h_g = 2
    exprs = [
        Eq(h, h_a + h_c + h_l),
        Eq(h_c, alpha * (K_st * N_P) / R),
        Eq(K_st, 2 / (k_s ** (-1) + k_t ** (-1))),
        Eq(R, np.sqrt(R_s ** 2 + R_t ** 2)),
        Eq(N_P, 1 - np.exp(- lambda_ * P / sigma_U)),
        Eq(h_l, beta * (K_stl * N_L) / R),
        Eq(K_stl, 3 / (k_s ** (-1) + k_t ** (-1) + k_l ** (-1))),
        Eq(N_L, 1 - np.exp(- gamma * delta)),
        Eq(h, h_c + h_g)
    ]

    k = 1
    p = 1
    H = 1
    θ = 1
    σ = 1
    K = 1
    C = 1
    A = 1
    B = 1
    k_f = 1
    k_w = 1
    h_f = 1

    exprs2 = [Eq(h, h_c + h_g), 
              Eq(h, 1.45*k*(p/H)**0.985*tan(θ)/σ), 
              Eq(h, 8000*λ*(K*p/(C*σ_U))**0.86), 
              Eq(h, A*(1 - exp(-B*P))), 
              Eq(h, 2*k_f*k_t*k_w*(1 - A)/(h_f*(-k_f*k_t - k_f*k_w + 2*k_t*k_w))), 
              Eq(h, h_a + h_c + h_l), 
              Eq(h_c, K_st*N_P*α/R), 
              Eq(K_st, 2/(1/k_t + 1/k_s)), 
              Eq(R, sqrt(Rs**2 + Rt**2)), 
              Eq(N_P, 1 - exp(-P*λ/σ_U)), 
              Eq(h_l, K_stl*N_L*β/R), 
              Eq(K_stl, 3/(1/k_t + 1/k_s + 1/k_l)), 
              Eq(N_L, 1 - exp(-γ*δ))]
    

    eg = EquationGraph(exprs2, 6)
    goodeqs = eg.get_system_of_equations()
    sol = solve(goodeqs)[0]
    sol2 = solve(eg.equations)[0]
    print(sol[h], sol2[h])
    # print(exprs)
    # print(goodeqs)
    

test()

14.2029991170233 14.2029991170233
